In [10]:
import os 
from google.colab import drive

In [11]:
!pip install tsmoothie

In [12]:
import numpy as np
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import tsmoothie
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from sklearn.decomposition import PCA
from tensorflow.keras import optimizers
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline

In [13]:
#LSTM=tf.compat.v1.keras.layers.CuDNNLSTM 
LSTM = tf.compat.v1.keras.layers.LSTM
GRU = tf.compat.v1.keras.layers.GRU
#GRU = tf.compat.v1.keras.layers.CuDNNGRU
a=tf.compat.v1.keras.layers.CuDNNLSTM
smoother=tsmoothie.KalmanSmoother(component='level_trend',  component_noise={'level':0.1, 'trend':0.1})

In [14]:
data_path = "EURUSD=X - EURUSD=X.csv"

In [15]:
# loading data

data_1=pd.read_csv(data_path)
data_1 = data_1.dropna()
print(data_1.columns)
data=data_1[['Open','High','Low','Close','Momentum','Average','Range','OHLC Price']]
dates=data_1[['Date']]

# normalising data

Open=MinMaxScaler()
High=MinMaxScaler()
Low=MinMaxScaler()
Close=MinMaxScaler()
Volume=MinMaxScaler()
Lowest=MinMaxScaler()

dataaa=np.zeros((data.shape),dtype=np.float64)
a=Open.fit_transform(data[['Open']])
dataaa[:,0] = a[:,0]
a=High.fit_transform(data[['High']])
dataaa[:,1] = a[:,0]
a=Low.fit_transform(data[['Low']])
dataaa[:,2] = a[:,0]
a=Close.fit_transform(data[['Close']])
dataaa[:,3] = a[:,0]
a=Volume.fit_transform(data[['Momentum']])
dataaa[:,4] = a[:,0]
a=Lowest.fit_transform(data[['Average']])
dataaa[:,5] = a[:,0]
a=Open.fit_transform(data[['Average']])
dataaa[:,6] = a[:,0]
a=Open.fit_transform(data[['Range']])
dataaa[:,7] = a[:,0]
a=Open.fit_transform(data[['OHLC Price']])
dataaa[:,7] = a[:,0]
a=dataaa[:,0]
smoother.smooth(a)
dataaa[:,0]=smoother.smooth_data

a=dataaa[:,1]
smoother.smooth(a)
dataaa[:,1]=smoother.smooth_data

a=dataaa[:,2]
smoother.smooth(a)
dataaa[:,2]=smoother.smooth_data

a=dataaa[:,3]
smoother.smooth(a)
dataaa[:,3]=smoother.smooth_data

a=dataaa[:,4]
smoother.smooth(a)
dataaa[:,4]=smoother.smooth_data

a=dataaa[:,5]
smoother.smooth(a)
dataaa[:,5]=smoother.smooth_data

a=dataaa[:,6]
smoother.smooth(a)
dataaa[:,6]=smoother.smooth_data

a=dataaa[:,7]
smoother.smooth(a)
dataaa[:,7]=smoother.smooth_data


full_data=dataaa

# finding total number of days

data=dataaa


total_days=data.shape[0]
print(total_days)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Momentum',
       'Average', 'Range', 'OHLC Price'],
      dtype='object')
4603


In [16]:
training_period_dates=dates.loc[:int(full_data.shape[0]*1)]
#testing_period_dates=dates.loc[int(full_data.shape[0]*0.9):]

In [56]:
total_days=data.shape[0]

timesteps=10

Train_data=data[:round(total_days*1),:]
#Test_data=data[round(total_days*0.9):,:]
print(Train_data.shape[0]," Days are used for Training")
#print(Test_data.shape[0]," Days are used for Testing")
# Function for ordering or making sequence of our test and train data according to date
def make_sequence(data,timesteps):
    X_train=[]
    Y_train=[]
    for i in range(len(data)-timesteps):
        X_train.append(data[i:(i+timesteps),[0,1,2,4,5,6,7]])
        Y_train.append(data[i+timesteps,3])
    return X_train,Y_train
Train_data,Train_Label=make_sequence(Train_data,timesteps)
Train_data,Train_Label=np.array(Train_data),np.array(Train_Label)
#Test_data,Test_Label=make_sequence(Test_data,timesteps)
#Test_data,Test_Label=np.array(Test_data),np.array(Test_Label)
input_shape=((Train_data).shape[1],(Train_data).shape[2])
print(input_shape)
Train_Label = Train_Label.reshape(-1,1)


4603  Days are used for Training
(10, 7)
[[0.28144503 0.28519541 0.7238577  0.53584218 0.47168658 0.47168658
  0.28321423]
 [0.28875654 0.29460366 0.73353664 0.512769   0.48128412 0.48128412
  0.29274589]
 [0.29700885 0.30261688 0.74130871 0.50931996 0.4895248  0.4895248
  0.30158356]
 [0.30396696 0.30954786 0.74656595 0.50955915 0.49581052 0.49581052
  0.30872379]
 [0.30968884 0.31595678 0.75032997 0.50089407 0.50112145 0.50112145
  0.31522292]
 [0.31589181 0.32155189 0.75316434 0.50579651 0.5056653  0.5056653
  0.32114871]
 [0.32114882 0.32590477 0.75492543 0.5146823  0.50894607 0.50894607
  0.32570132]
 [0.32475666 0.32897803 0.75577586 0.52068631 0.51097563 0.51097563
  0.32872376]
 [0.32687889 0.33198445 0.75654891 0.50896161 0.51294356 0.51294356
  0.33162586]
 [0.32979432 0.33650789 0.75800394 0.49491641 0.51614184 0.51614184
  0.33585915]]


In [53]:
#model building
model=Sequential()

#adding layers to the model
model.add(GRU(20,input_shape=input_shape,return_sequences=True))
model.add(LSTM(256,return_sequences = True))
model.add(Dense(64))
model.add(Dense(1))

#compiling model
lr=1e-3
model.compile(optimizer = optimizers.Adam(learning_rate= lr), loss = 'mean_squared_error')

#fitting model
history=model.fit(Train_data,Train_Label,batch_size=36,epochs=60)

Epoch 1/60
128/128 [==============================] - 15s 83ms/step - loss: 0.0116
Epoch 2/60
128/128 [==============================] - 11s 83ms/step - loss: 0.0012
Epoch 3/60
128/128 [==============================] - 10s 82ms/step - loss: 6.9628e-04
Epoch 4/60
128/128 [==============================] - 11s 82ms/step - loss: 6.4857e-04
Epoch 5/60
128/128 [==============================] - 11s 82ms/step - loss: 6.4430e-04
Epoch 6/60
128/128 [==============================] - 11s 82ms/step - loss: 6.3102e-04
Epoch 7/60
128/128 [==============================] - 11s 82ms/step - loss: 6.3068e-04
Epoch 8/60
128/128 [==============================] - 11s 84ms/step - loss: 6.3190e-04
Epoch 9/60
128/128 [==============================] - 11s 83ms/step - loss: 6.5306e-04
Epoch 10/60
128/128 [==============================] - 11s 83ms/step - loss: 6.2192e-04
Epoch 11/60
128/128 [==============================] - 11s 83ms/step - loss: 6.9853e-04
Epoch 12/60
128/128 [============================

In [24]:
testing_period_dates=testing_period_dates[10:]
a=testing_period_dates['Date'].loc[:]

NameError: ignored

In [54]:
#predicting values from our model of test data
Test_Predicted_Label=model.predict(Train_data)
print(Test_Predicted_Label.shape)
print(Test_Predicted_Label)
Test_Label = Train_Label.reshape(Train_Label.shape[0],1)
y_hat_test = Close.inverse_transform(Test_Predicted_Label)
#y_test = Close.inverse_transform(Test_Label)
np.savetxt("predicted.csv", y_hat_test, delimiter=",")
#calculating and printing mean square error
mse = mean_squared_error(y_test,y_hat_test)
print("Mean Square Error of Test Data is ",mse)
ratio_mean_predicted = np.mean(y_hat_test / y_test)
ratio_std_predicted  = np.std(y_hat_test / y_test)

(4593, 10, 1)
[[[0.29315162]
  [0.3238381 ]
  [0.32437074]
  ...
  [0.341089  ]
  [0.35873532]
  [0.36474603]]

 [[0.3139323 ]
  [0.32600623]
  [0.32996964]
  ...
  [0.35892618]
  [0.36481506]
  [0.37166363]]

 [[0.324387  ]
  [0.33054984]
  [0.34395266]
  ...
  [0.36496204]
  [0.37165534]
  [0.36543006]]

 ...

 [[0.26014373]
  [0.25742817]
  [0.25715888]
  ...
  [0.24200423]
  [0.23916282]
  [0.23763129]]

 [[0.25854516]
  [0.25629503]
  [0.25488687]
  ...
  [0.23947082]
  [0.23778662]
  [0.23592432]]

 [[0.2572503 ]
  [0.254268  ]
  [0.2519654 ]
  ...
  [0.23809613]
  [0.2360823 ]
  [0.23219593]]]


ValueError: ignored

In [ ]:
# plotting actual vs predicted values price
plt.style.use('default')
plt.figure(figsize=(13,4))
plt.plot(y_test,c = 'r')
plt.plot(y_hat_test,c = 'y')
plt.xticks(range(0,a.shape[0],100),a.loc[::100],rotation=45)
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('proceedit 20210122 - SPP - Case 2-Stock Prediction Graph using Multivariate-LSTM model')
plt.legend(['Actual','Predicted'],loc = 'lower right')

In [ ]:
# plotting offset by date
plt.style.use('default')

plt.figure(figsize=(15,5))
plt.plot((y_test-y_hat_test))
plt.xticks(range(0,a.shape[0],100),a.loc[::100],rotation=45)
plt.grid(True)

plt.xlabel('Date')
plt.ylabel('Offset')
plt.rc('axes', titlesize=20) 
plt.rc('axes', labelsize=15)
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.title('proceedit 20210122 - SPP - Case 2- Offset=(Predicted-Actual)/Actual vs Days')

In [ ]:
# plotting ratio of Predicted vs Actual Price
plt.style.use('default')
plt.figure(figsize=(15,5))
plt.plot((y_test/y_hat_test))
plt.xticks(range(0,a.shape[0],100),a.loc[::100],rotation=45)
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Ratio of Predicted and Actual Price')
plt.rc('axes', titlesize=20) 
plt.rc('axes', labelsize=15)
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.title('proceedit 20210122 - SPP - Case 2- Ratio of Predicted and Actual Price vs Days')
plt.style.use('default')

In [ ]:
# plotting actual close price vs the ratio between predicted and actual price
plt.figure(figsize=(10,5))

plt.scatter(y_test,y_hat_test/y_test)
m, b = np.polyfit(y_test[:,0],y_hat_test/y_test[:,0], 1)
plt.title('proceedit 20210116- SPP - Case 2 - x representing the Actual Close prices and in y the ratio between Predicted and Actual price')
plt.plot(y_test[:,0], m*y_test[:,0]+ b,color='red')
plt.xlabel('True Price')
plt.ylabel('Predicted Price')
plt.grid(True)
r2=r2_score(y_test,y_hat_test)
print("R2 Score is ",r2)

In [ ]:
# predicted values from model
Train_Predicted_Label=model.predict(Train_data)
Train_Test_Label=Train_Label.reshape(Train_Label.shape[0],1)

# inverse transformation
Train_y_hat=Lowest.inverse_transform(Train_Predicted_Label)
Train_y=Lowest.inverse_transform(Train_Test_Label)

In [ ]:
mse=mean_squared_error(Train_y,Train_y_hat)
print(mse)

In [ ]:
# calculating ratio and printing
ratio_mean_Train=np.mean(Train_y_hat/Train_y)
ratio_std_Train=np.std(Train_y_hat/Train_y)

In [ ]:
model.save(path+'S&P500_OHLCVM_model.h5')

In [ ]:
os.listdir(path)
print(path)
# saving our predicted values in .csv file
np.savetxt(path+'train_OHLCVM_10_2.csv',Train_y_hat)
np.savetxt(path+'test_OHLCVM_10_2.csv',y_hat_test)

In [ ]:
# saving our predicted values in .csv file
np.savetxt(path+'train_OHLCVM_10_2.csv',Train_y_hat)
np.savetxt(path+'test_OHLCVM_10_2.csv',y_hat_test)